This notebook contains the code for the meta-analysis of healthy lung data for ACE2, TMPRSS2, and CTSL. It contains the complex model with interaction terms that was run on the cell-level data (not pseudo-bulk, no holdout analysis).

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

/home/icb/malte.luecken/anaconda3/envs/cov19_py/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

scanpy==1.4.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


'v0.7.3'

In [3]:
#User inputs
folder = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study'

adata_diffxpy = '/storage/groups/ml01/workspace/malte.luecken/2020_cov19_study/COVID19_lung_atlas_revision_v3.h5ad'

output_folder = 'diffxpy_out/'

de_output_base = 'COVID19_lung_atlas_revision_v3_lung_cov19_poissonglm_smoking_nUMIoffset_testInts'

# Read the data

In [4]:
adata = sc.read(adata_diffxpy)

In [5]:
adata

AnnData object with n_obs × n_vars = 1320896 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [6]:
adata.obs.age = adata.obs.age.astype(float)

In [7]:
adata.obs.dtypes

age                         float64
anatomical_region          category
donor                      category
last_author/PI             category
lung_vs_nasal              category
notes                      category
original_celltype_ann      category
sample                     category
sex                        category
smoking                    category
total_counts                float64
smoked_boolean             category
last_author_sample_name    category
ann_level_1                category
ann_level_2                category
ann_level_3                category
ann_level_4                category
ann_level_5                category
ann_highest_res               int64
ann_new                        bool
dtype: object

In [8]:
adata.obs['dataset'] = adata.obs['last_author/PI']

In [9]:
adata.obs.dataset.value_counts()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    95303
Spence                      78401
Barbry/Leroy                76981
Krasnow/Quake               60993
Meyer                       57020
Rawlins                     53409
Regev                       43527
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Misharin                    28329
Xavier/Regev                25552
Spira/Campbell              24455
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Schultze                    22641
Schiller                    20776
Nawijn                      18197
Teichmann                   12971
Kropski/Banovich_dnar        8359
Shalek                       7603
Linnarsson                   4640
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      2207
Schultze/Falk                1965
Beane         

In [10]:
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()
adata.obs['dataset'].nunique()

377

228

31

# Filter the data

Keep only datsets with:
- more than 1 donor
- non-fetal
- lung

In [11]:
# Remove fetal datasets
dats_to_remove = set(['Rawlins', 'Spence', 'Linnarsson'])

In [12]:
dat = adata.obs.groupby(['donor']).agg({'sex':'first', 'age':'first', 'dataset':'first'})

# Single donor filter
don_tab = dat['dataset'].value_counts()
dats_to_remove.update(set(don_tab.index[don_tab == 1]))

In [13]:
dats_to_remove = list(dats_to_remove)
dats_to_remove

['Misharin', 'Schultze/Falk', 'Linnarsson', 'Spence', 'Rawlins']

In [14]:
adata = adata[~adata.obs.dataset.isin(dats_to_remove)].copy()

In [15]:
adata.obs.lung_vs_nasal.value_counts()

lung     1096604
nasal      57548
Name: lung_vs_nasal, dtype: int64

In [16]:
# Filter for only lung data
adata = adata[adata.obs.lung_vs_nasal.isin(['lung']),].copy()

In [17]:
adata

AnnData object with n_obs × n_vars = 1096604 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset'

In [18]:
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()
adata.obs['dataset'].nunique()

309

185

24

# Binarize smoking status

In [19]:
adata.obs.smoking.value_counts()

never              575345
current            173020
nan                111184
former              53487
non-smoker          42367
never-smoker        33119
active              29396
smoked              28197
current/former      23573
heavy               15663
current/former?      5162
light                3316
Former               1385
Current               610
Never Smoker          426
Current Smoker        354
Name: smoking, dtype: int64

In [20]:
adata.obs['smoking_status'] = adata.obs.smoked_boolean
#adata.obs['smoking_status'] = [True if stat in ['current', 'smoked', 'active', 'former', 'heavy', 'light'] else False if stat in ['never', 'nonsmoker'] else "nan" for stat in adata.obs.smoking]

In [21]:
adata.obs.smoking_status.value_counts()

False    651257
True     334163
nan      111184
Name: smoking_status, dtype: int64

## Filter out data w/o smoking status

In [22]:
pd.crosstab(adata.obs.dataset, adata.obs.smoking)

smoking,Current,Current Smoker,Former,Never Smoker,active,current,current/former,current/former?,former,heavy,light,nan,never,never-smoker,non-smoker,smoked
dataset,,,,,,,,,,,,,,,,
Barbry/Leroy,0,0,0,0,0,0,0,0,16423,0,0,0,0,0,42367,0
Beane,0,0,0,0,0,305,0,0,581,0,0,0,0,0,0,0
Beane/Campbell,0,354,0,426,0,0,0,0,0,0,0,0,0,0,0,0
Eils/Conrad/Kreuter,0,0,0,0,0,11185,0,0,0,0,0,0,28593,0,0,0
Kaminski,0,0,0,0,0,0,23573,0,0,0,0,3850,67880,0,0,0
Koenigshoff,0,0,0,0,0,0,0,0,0,0,0,0,0,33119,0,0
Krasnow/Quake,0,0,0,0,0,0,0,0,7524,0,0,0,53469,0,0,0
Kropski/Banovich_dnar,0,0,0,0,0,0,0,0,0,0,0,0,2647,0,0,5712
Kropski/Banovich_vand,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,22485


In [23]:
adata = adata[~adata.obs.smoking_status.isin(['nan']),].copy()
adata

AnnData object with n_obs × n_vars = 985420 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

In [24]:
adata.obs.dataset.value_counts()
adata.obs['sample'].nunique()
adata.obs['donor'].nunique()

Regev/Rajagopal            322998
Meyer_b                    117535
Kaminski                    91453
Krasnow/Quake               60993
Barbry/Leroy                58790
Meyer                       57020
Misharin/Budinger           41266
Eils/Conrad/Kreuter         39778
Seibold                     36248
Whitsett/Xu_10X             34185
Koenigshoff                 33119
Lafyatis/Rojas              24220
Kropski/Banovich_vand       23285
Teichmann                   12971
Nawijn                      11110
Kropski/Banovich_dnar        8359
Schiller                     5162
Whitsett/Xu_dropSeq          3267
Mazzilli/Campbell/Beane      1995
Beane                         886
Beane/Campbell                780
Name: dataset, dtype: int64

286

164

# Check the data

In [25]:
np.mean(adata.X.astype(int) != adata.X)

0.0

In [26]:
# Check if any non-integer data in a particular dataset
for dat in adata.obs.dataset.unique():
    val = np.mean(adata[adata.obs.dataset.isin([dat]),:].X.astype(int) != adata[adata.obs.dataset.isin([dat]),:].X)
    if val != 0:
        print(f'dataset= {dat}; value= {val}')
        adata[adata.obs.dataset.isin([dat]),:].X[:20,:20].A

All counts are integers, or are very close to integers!

In [27]:
adata.obs.age.value_counts()
adata.obs.sex.value_counts()

57.00    81833
66.00    69371
42.00    66596
59.00    59955
18.00    59849
46.00    56291
64.00    46130
35.00    45869
0.25     29908
67.50    28201
20.00    25362
51.00    24766
30.00    24095
29.00    19248
0.00     19065
32.00    19009
3.00     18387
42.50    16906
41.00    14943
58.00    14086
57.50    13840
23.00    13657
68.00    11852
65.00    11404
56.00    11024
75.00    10941
45.00    10939
55.00    10878
47.00    10278
49.00    10198
21.00     9430
27.00     8693
72.50     7983
22.00     7980
63.00     7600
31.00     7085
33.00     6906
44.00     6681
52.50     6326
38.00     5629
62.00     4679
24.00     4433
62.50     4073
26.00     4066
61.00     3922
67.00     3916
50.00     3831
32.50     3772
80.00     3261
10.00     2694
40.00     2647
17.00     2552
79.00     2550
54.00     2507
76.00     2447
36.00     2218
48.00     1153
25.00      823
70.00      168
69.00      130
87.00      112
43.00       79
34.00       76
78.00       65
74.00       52
Name: age, dtype: int64

female    550340
male      435080
Name: sex, dtype: int64

# Fit models and perform DE

In [28]:
cluster_key = 'ann_level_2'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Myeloid',
 'Airway epithelium',
 'Alveolar epithelium',
 'Lymphoid',
 'Fibroblast lineage',
 'Blood vessels',
 'Submucosal Gland',
 'Smooth Muscle',
 'Lymphatics',
 'Mesothelium']

Calculate DE genes per cluster.

In [29]:
adata

AnnData object with n_obs × n_vars = 985420 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status'

In [30]:
adata.obs['total_counts_scaled'] = adata.obs['total_counts']/adata.obs['total_counts'].mean()

In [31]:
# Get interquartile range for ages to test
adata.obs.groupby(['donor']).agg({'age':'first'}).age.quantile([0.25,0.5,0.75])

0.25    30.75
0.50    49.50
0.75    62.50
Name: age, dtype: float64

In [32]:
formula = "1 + sex + age + sex:age + smoking_status + sex:smoking_status + age:smoking_status + dataset"
tested_coef = ["sex[T.male]", "age", "smoking_status[T.True]"]
dmat = de.utils.design_matrix(
    data=adata,
    formula="~" + formula,
    as_numeric=["age"],
    return_type="patsy"
)

to_test = dict()
to_test['age'] = [31,62]
to_test['sex[T.male]'] = [0,1]
to_test['smoking_status[T.True]'] = [0,1]

dmat[1]

['Intercept',
 'sex[T.male]',
 'smoking_status[T.True]',
 'dataset[T.Beane]',
 'dataset[T.Beane/Campbell]',
 'dataset[T.Eils/Conrad/Kreuter]',
 'dataset[T.Kaminski]',
 'dataset[T.Koenigshoff]',
 'dataset[T.Krasnow/Quake]',
 'dataset[T.Kropski/Banovich_dnar]',
 'dataset[T.Kropski/Banovich_vand]',
 'dataset[T.Lafyatis/Rojas]',
 'dataset[T.Mazzilli/Campbell/Beane]',
 'dataset[T.Meyer]',
 'dataset[T.Meyer_b]',
 'dataset[T.Misharin/Budinger]',
 'dataset[T.Nawijn]',
 'dataset[T.Regev/Rajagopal]',
 'dataset[T.Schiller]',
 'dataset[T.Seibold]',
 'dataset[T.Teichmann]',
 'dataset[T.Whitsett/Xu_10X]',
 'dataset[T.Whitsett/Xu_dropSeq]',
 'sex[T.male]:smoking_status[T.True]',
 'age',
 'sex[T.male]:age',
 'age:smoking_status[T.True]']

### Function definition to test effect sizes at particular covariate values

In [33]:
def calc_effects(dmat, cov_mat, params, effect, coefs):
    from patsy.design_info import DesignMatrix
    from diffxpy.api.stats import wald_test_chisq
    
    dmat_cond = isinstance(dmat, tuple) and isinstance(dmat[0], DesignMatrix)

    if not dmat_cond:
        raise ValueError("`dmat` should be a patsy output Design Matrix.")

    effect_list = ['sex[T.male]', 'age', 'smoking_status[T.True]']
    if not effect in effect_list:
        raise ValueError(f'{effect} is not one of: '
                         f'{effect_list}')

    if not isinstance(coefs, dict):
        raise TypeError('`coefs` should contain a dictionary of coefficients '
                        'where the effects should be evaluated.')
    
    ## Note: this is only correct when 3 covariates are tested in combinations
    #if np.sum([coef in coefs for coef in effect_list]) < 2:
    #    raise ValueError('The `coefs` dict must contain values for the two '
    #                     'coefficient not tested in:'
    #                     f'{effect_list}')
        
    if 'smoking_status[T.True]' in coefs and coefs['smoking_status[T.True]'] not in [0,1]:
        raise ValueError('Smoking status should be encoded as 0 or 1.')

    if 'sex[T.male]' in coefs and coefs['sex[T.male]'] not in [0,1]:
        raise ValueError('Sex should be encoded as 0 or 1.')

    if 'age' in coefs and not (isinstance(coefs['age'], float) or isinstance(coefs['age'], int)):
        raise ValueError('Age should be a numerical value.')

    coef_list = []

    for term in dmat[1]:
        if effect not in term:
            coef_list.append(0)
        elif term == effect:
            coef_list.append(1)
        else:
            t_list = term.split(':')
            t_list.remove(effect)
            coef_list.append(coefs[t_list[0]])

    C = np.array(coef_list)
    val = np.matmul(C,np.array(params))
    stderr = np.sqrt(np.matmul(np.matmul(C.T,cov_mat),C))
    pval = wald_test_chisq(np.array([val]).reshape(1,1), np.array([stderr**2]).reshape(1,1,1))[0]
    return (val, stderr, pval)

## Poisson GLM

In [34]:
# Poisson GLM loop
de_results_lvl2_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata[adata.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print(f'Testing in {adata_tmp.n_obs} cells...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp,
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0], 
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                for c2 in to_test[iter_coefs[1]]:
                    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                    val, stderr, pval = calc_effects(
                        dmat = dmat,
                        cov_mat = cov_mat,
                        params = pois_results.params,
                        effect = coef,
                        coefs = coef_vals)
                    
                    case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                    case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                    case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                    case = case.replace('age:31','31yr').replace('age:62','62yr')
                    case = case.replace('_',' ')
                    
                    # Output the results nicely
                    de_results_temp = pd.DataFrame({
                        "gene": gene,
                        "cell_identity": clust,
                        "covariate": coef,
                        "eval_at": case,
                        "coef": val,
                        "coef_sd": stderr,                 
                        "pval": pval
                    }, index= [clust+"_"+gene+"_"+coef])
                    
                    de_results_list.append(de_results_temp)
                    
    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm = pd.concat([de_results_lvl2_glm[i] for i in de_results_lvl2_glm.keys()], ignore_index=True)

In cluster Myeloid:
False    137441
True     109516
Name: smoking_status, dtype: int64
male      130070
female    116887
Name: sex, dtype: int64
Testing 3 genes...
Testing in 246957 cells...

In cluster Airway epithelium:
False    157085
True      61702
Name: smoking_status, dtype: int64
female    125772
male       93015
Name: sex, dtype: int64
Testing 3 genes...
Testing in 218787 cells...

In cluster Alveolar epithelium:
False    124700
True      60785
Name: smoking_status, dtype: int64
female    123154
male       62331
Name: sex, dtype: int64
Testing 3 genes...
Testing in 185485 cells...

In cluster Lymphoid:
False    84562
True     48215
Name: smoking_status, dtype: int64
female    82702
male      50075
Name: sex, dtype: int64
Testing 3 genes...
Testing in 132777 cells...

In cluster Fibroblast lineage:
False    40012
True     13154
Name: smoking_status, dtype: int64
female    31351
male      21815
Name: sex, dtype: int64
Testing 3 genes...
Testing in 53166 cells...

In cluster Bloo

In [35]:
# Also test how the model behaves without adding the smoking coviarate
formula_nosmoke = "1 + sex + age + sex:age + dataset"
tested_coef_nosmoke = ["sex[T.male]", "age"]


In [36]:
# Poisson GLM loop
de_results_lvl2_glm_nosmoke = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata[adata.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula_nosmoke,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            exog=dmat[0], 
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef_nosmoke:
            iter_coefs_nosmoke = tested_coef_nosmoke.copy()
            iter_coefs_nosmoke.remove(coef)
            
            for c1 in to_test[iter_coefs_nosmoke[0]]:
                coef_vals = {iter_coefs_nosmoke[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                case = '_'.join([iter_coefs_nosmoke[0]+':'+str(c1)])
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:31','31yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)
                
                
        # Test over coefs
        for coef in tested_coef_nosmoke:
            de_results_temp = pois_results.wald_test(
                [x for i, x in enumerate(pois_model.exog_names) if dmat[1][i] in [coef]]
            )

            # Output the results nicely
            de_results_temp = pd.DataFrame({
                "gene": gene,
                "cell_identity": clust,
                "covariate": coef,
                "coef": pois_results.params[[y == coef for y in dmat[1]]],
                "coef_sd": pois_results.bse[[y == coef for y in dmat[1]]],                 
                "pval": de_results_temp.pvalue
            }, index= [clust+"_"+gene+"_"+coef])

            de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl2_glm_nosmoke[clust] = de_results
    
# Join the dataframes:
full_res_lvl2_glm_nosmoke = pd.concat([de_results_lvl2_glm_nosmoke[i] for i in de_results_lvl2_glm_nosmoke.keys()], ignore_index=True)

In cluster Myeloid:
False    137441
True     109516
Name: smoking_status, dtype: int64
male      130070
female    116887
Name: sex, dtype: int64
Testing 3 genes...

In cluster Airway epithelium:
False    157085
True      61702
Name: smoking_status, dtype: int64
female    125772
male       93015
Name: sex, dtype: int64
Testing 3 genes...

In cluster Alveolar epithelium:
False    124700
True      60785
Name: smoking_status, dtype: int64
female    123154
male       62331
Name: sex, dtype: int64
Testing 3 genes...

In cluster Lymphoid:
False    84562
True     48215
Name: smoking_status, dtype: int64
female    82702
male      50075
Name: sex, dtype: int64
Testing 3 genes...

In cluster Fibroblast lineage:
False    40012
True     13154
Name: smoking_status, dtype: int64
female    31351
male      21815
Name: sex, dtype: int64
Testing 3 genes...

In cluster Blood vessels:
False    21455
True     21064
Name: smoking_status, dtype: int64
male      31078
female    11441
Name: sex, dtype: int64
Te

## Inspect some results

In [37]:
de_results_lvl2_glm.keys()

dict_keys(['Myeloid', 'Airway epithelium', 'Alveolar epithelium', 'Lymphoid', 'Fibroblast lineage', 'Blood vessels', 'Submucosal Gland', 'Smooth Muscle', 'Lymphatics', 'Mesothelium'])

In [38]:
full_res_lvl2_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],31yr NS,1.073540,0.298107,3.167587e-04,7.127070e-04
1,ACE2,Myeloid,sex[T.male],31yr S,0.146963,0.519140,7.771087e-01,7.771087e-01
2,ACE2,Myeloid,sex[T.male],62yr NS,0.174235,0.342187,6.106254e-01,6.280719e-01
3,ACE2,Myeloid,sex[T.male],62yr S,-0.752343,0.444831,9.077924e-02,1.361689e-01
4,ACE2,Myeloid,age,F NS,0.013306,0.008423,1.141558e-01,1.580618e-01
5,ACE2,Myeloid,age,F S,0.041732,0.019726,3.438045e-02,5.625891e-02
6,ACE2,Myeloid,age,M NS,-0.015704,0.011066,1.558627e-01,1.934847e-01
7,ACE2,Myeloid,age,M S,0.012722,0.017144,4.580572e-01,4.996988e-01
8,ACE2,Myeloid,smoking_status[T.True],F 31yr,-0.301265,0.521503,5.634766e-01,5.966223e-01
9,ACE2,Myeloid,smoking_status[T.True],F 62yr,0.579939,0.384322,1.313008e-01,1.688154e-01


In [39]:
full_res_lvl2_glm.loc[full_res_lvl2_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],31yr NS,1.073540,0.298107,3.167587e-04,7.127070e-04
1,ACE2,Myeloid,sex[T.male],31yr S,0.146963,0.519140,7.771087e-01,7.771087e-01
2,ACE2,Myeloid,sex[T.male],62yr NS,0.174235,0.342187,6.106254e-01,6.280719e-01
3,ACE2,Myeloid,sex[T.male],62yr S,-0.752343,0.444831,9.077924e-02,1.361689e-01
4,ACE2,Myeloid,age,F NS,0.013306,0.008423,1.141558e-01,1.580618e-01
5,ACE2,Myeloid,age,F S,0.041732,0.019726,3.438045e-02,5.625891e-02
6,ACE2,Myeloid,age,M NS,-0.015704,0.011066,1.558627e-01,1.934847e-01
7,ACE2,Myeloid,age,M S,0.012722,0.017144,4.580572e-01,4.996988e-01
8,ACE2,Myeloid,smoking_status[T.True],F 31yr,-0.301265,0.521503,5.634766e-01,5.966223e-01
9,ACE2,Myeloid,smoking_status[T.True],F 62yr,0.579939,0.384322,1.313008e-01,1.688154e-01


In [40]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'ACE2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Myeloid,sex[T.male],31yr NS,1.073540,0.298107,3.167587e-04,7.127070e-04
10,ACE2,Myeloid,smoking_status[T.True],M 31yr,-1.227843,0.492286,1.262535e-02,2.164347e-02
36,ACE2,Airway epithelium,sex[T.male],31yr NS,0.169050,0.054561,1.945928e-03,2.415635e-03
37,ACE2,Airway epithelium,sex[T.male],31yr S,0.255041,0.069809,2.587653e-04,3.726221e-04
40,ACE2,Airway epithelium,age,F NS,0.008869,0.001747,3.861422e-07,6.043965e-07
41,ACE2,Airway epithelium,age,F S,-0.007782,0.002456,1.533047e-03,1.971060e-03
42,ACE2,Airway epithelium,age,M NS,0.003985,0.001433,5.420012e-03,6.504015e-03
43,ACE2,Airway epithelium,age,M S,-0.012666,0.002478,3.212723e-07,5.257183e-07
44,ACE2,Airway epithelium,smoking_status[T.True],F 31yr,0.429234,0.074092,6.902755e-09,1.183329e-08
46,ACE2,Airway epithelium,smoking_status[T.True],M 31yr,0.515224,0.071545,5.958567e-13,1.430056e-12


In [41]:
full_res_lvl2_glm.loc[(full_res_lvl2_glm['gene'] == 'TMPRSS2') & (full_res_lvl2_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
12,TMPRSS2,Myeloid,sex[T.male],31yr NS,0.690563,0.070619,0.000000e+00,0.000000e+00
14,TMPRSS2,Myeloid,sex[T.male],62yr NS,0.907187,0.075332,0.000000e+00,0.000000e+00
17,TMPRSS2,Myeloid,age,F S,-0.014415,0.004841,2.906770e-03,5.813540e-03
18,TMPRSS2,Myeloid,age,M NS,0.009990,0.002973,7.787665e-04,1.649153e-03
20,TMPRSS2,Myeloid,smoking_status[T.True],F 31yr,0.293402,0.100582,3.533628e-03,6.695295e-03
21,TMPRSS2,Myeloid,smoking_status[T.True],F 62yr,-0.246515,0.092834,7.920702e-03,1.425726e-02
22,TMPRSS2,Myeloid,smoking_status[T.True],M 31yr,-0.495351,0.126897,9.478403e-05,2.274817e-04
23,TMPRSS2,Myeloid,smoking_status[T.True],M 62yr,-1.035268,0.103331,0.000000e+00,0.000000e+00
48,TMPRSS2,Airway epithelium,sex[T.male],31yr NS,0.274595,0.015938,0.000000e+00,0.000000e+00
49,TMPRSS2,Airway epithelium,sex[T.male],31yr S,0.351465,0.023426,0.000000e+00,0.000000e+00


# Level 3 annotation

In [42]:
cluster_key = 'ann_level_3'
clust_tbl = adata.obs[cluster_key].value_counts()
clusters = clust_tbl.index[clust_tbl > 1000]
ct_to_rm = clusters[[ct.startswith('1') or ct.startswith('2') for ct in clusters]]
clusters = clusters.drop(ct_to_rm.tolist()).tolist()
clusters

['Macrophages',
 'Basal',
 'AT2',
 'T cell lineage',
 'Monocytes',
 'Multiciliated lineage',
 'Submucosal Secretory',
 'AT1',
 'Innate lymphoid cells',
 'Secretory',
 'Capillary',
 'Mast cells',
 'B cell lineage',
 'Fibroblasts',
 'Dendritic cells',
 'Venous',
 'Lymphatic EC',
 'Arterial',
 'Rare',
 'Airway smooth muscle']

In [43]:
adata_sub = adata[adata.obs.ann_level_3.isin(clusters),:]

adata_sub
adata_sub.obs.donor.nunique()
adata_sub.obs['sample'].nunique()

View of AnnData object with n_obs × n_vars = 795195 × 3 
    obs: 'age', 'anatomical_region', 'donor', 'last_author/PI', 'lung_vs_nasal', 'notes', 'original_celltype_ann', 'sample', 'sex', 'smoking', 'total_counts', 'smoked_boolean', 'last_author_sample_name', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'dataset', 'smoking_status', 'total_counts_scaled'

164

286

## Poisson GLM

In [44]:
# Poisson GLM loop
de_results_lvl3_glm = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata_sub[adata_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print(f'Testing in {adata_tmp.n_obs} cells...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0],
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef:
            iter_coefs = tested_coef.copy()
            iter_coefs.remove(coef)
            
            for c1 in to_test[iter_coefs[0]]:
                for c2 in to_test[iter_coefs[1]]:
                    coef_vals = {iter_coefs[0]:c1, iter_coefs[1]:c2}
                    val, stderr, pval = calc_effects(
                        dmat = dmat,
                        cov_mat = cov_mat,
                        params = pois_results.params,
                        effect = coef,
                        coefs = coef_vals)
                    
                    case = '_'.join([iter_coefs[0]+':'+str(c1), iter_coefs[1]+':'+str(c2)])
                    case = case.replace('smoking_status[T.True]:0','NS').replace('smoking_status[T.True]:1','S')
                    case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                    case = case.replace('age:31','31yr').replace('age:62','62yr')
                    case = case.replace('_',' ')
                    
                    # Output the results nicely
                    de_results_temp = pd.DataFrame({
                        "gene": gene,
                        "cell_identity": clust,
                        "covariate": coef,
                        "eval_at": case,
                        "coef": val,
                        "coef_sd": stderr,                 
                        "pval": pval
                    }, index= [clust+"_"+gene+"_"+coef])
                    
                    de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm = pd.concat([de_results_lvl3_glm[i] for i in de_results_lvl3_glm.keys()], ignore_index=True)

In cluster Macrophages:
False    84537
True     72427
Name: smoking_status, dtype: int64
male      94376
female    62588
Name: sex, dtype: int64
Testing 3 genes...
Testing in 156964 cells...

In cluster Basal:
False    113299
True      42578
Name: smoking_status, dtype: int64
female    91571
male      64306
Name: sex, dtype: int64
Testing 3 genes...
Testing in 155877 cells...

In cluster AT2:
False    100593
True      54919
Name: smoking_status, dtype: int64
female    103012
male       52500
Name: sex, dtype: int64
Testing 3 genes...
Testing in 155512 cells...

In cluster T cell lineage:
True     26540
False    25599
Name: smoking_status, dtype: int64
female    27637
male      24502
Name: sex, dtype: int64
Testing 3 genes...
Testing in 52139 cells...

In cluster Monocytes:
True     23487
False    19216
Name: smoking_status, dtype: int64
female    21616
male      21087
Name: sex, dtype: int64
Testing 3 genes...
Testing in 42703 cells...

In cluster Multiciliated lineage:
False    26913


In [45]:
# Poisson GLM loop
de_results_lvl3_glm_nosmoke = dict()

# Test over clusters
for clust in clusters:
    adata_tmp = adata_sub[adata_sub.obs[cluster_key] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs['smoking_status'].value_counts())
    print(adata_tmp.obs['sex'].value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=10)
    if adata_tmp.n_vars == 0:
        print('No genes expressed in more than 10 cells!')
        continue
    if len(adata_tmp.obs.smoking_status.value_counts())==1:
        print(f'{clust} only has 1 type of smoker/nonsmoker sample.')
        continue
        
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")

    # List to store results
    de_results_list = []        

    # Set up design matrix
    dmat = de.utils.design_matrix(
        data=adata_tmp, #[idx_train],
        formula="~" + formula_nosmoke,
        as_numeric=["age"],
        return_type="patsy"
    )
    
    # Test if model is full rank
    if np.linalg.matrix_rank(np.asarray(dmat[0])) < np.min(dmat[0].shape):
        print(f'Cannot test {clust} as design matrix is not full rank.')
        continue
    
    for i, gene in enumerate(adata_tmp.var_names):
        # Specify model
        pois_model = sm.GLM(
            endog=adata_tmp.X[:, i].todense(), #[idx_train, :], 
            exog=dmat[0], 
            offset=np.log(adata_tmp.obs['total_counts_scaled'].values),
            family=sm.families.Poisson()
        )

        # Fit the model
        pois_results = pois_model.fit()

        # Get the covariance matrix
        cov_mat = pois_results.cov_params()

        # Test over coefs
        for coef in tested_coef_nosmoke:
            iter_coefs_nosmoke = tested_coef_nosmoke.copy()
            iter_coefs_nosmoke.remove(coef)
            
            for c1 in to_test[iter_coefs_nosmoke[0]]:
                coef_vals = {iter_coefs_nosmoke[0]:c1}
                val, stderr, pval = calc_effects(
                    dmat = dmat,
                    cov_mat = cov_mat,
                    params = pois_results.params,
                    effect = coef,
                    coefs = coef_vals)
                
                case = '_'.join([iter_coefs_nosmoke[0]+':'+str(c1)])
                case = case.replace('sex[T.male]:0','F').replace('sex[T.male]:1','M')
                case = case.replace('age:31','31yr').replace('age:62','62yr')
                case = case.replace('_',' ')
                
                # Output the results nicely
                de_results_temp = pd.DataFrame({
                    "gene": gene,
                    "cell_identity": clust,
                    "covariate": coef,
                    "eval_at": case,
                    "coef": val,
                    "coef_sd": stderr,                 
                    "pval": pval
                }, index= [clust+"_"+gene+"_"+coef])
                
                de_results_list.append(de_results_temp)

    de_results = pd.concat(de_results_list)
    de_results['adj_pvals'] = multipletests(de_results['pval'].tolist(), method='fdr_bh')[1]
    
    # Store the results
    de_results_lvl3_glm_nosmoke[clust] = de_results
    
# Join the dataframes:
full_res_lvl3_glm_nosmoke = pd.concat([de_results_lvl3_glm_nosmoke[i] for i in de_results_lvl3_glm_nosmoke.keys()], ignore_index=True)

In cluster Macrophages:
False    84537
True     72427
Name: smoking_status, dtype: int64
male      94376
female    62588
Name: sex, dtype: int64
Testing 3 genes...

In cluster Basal:
False    113299
True      42578
Name: smoking_status, dtype: int64
female    91571
male      64306
Name: sex, dtype: int64
Testing 3 genes...

In cluster AT2:
False    100593
True      54919
Name: smoking_status, dtype: int64
female    103012
male       52500
Name: sex, dtype: int64
Testing 3 genes...

In cluster T cell lineage:
True     26540
False    25599
Name: smoking_status, dtype: int64
female    27637
male      24502
Name: sex, dtype: int64
Testing 3 genes...

In cluster Monocytes:
True     23487
False    19216
Name: smoking_status, dtype: int64
female    21616
male      21087
Name: sex, dtype: int64
Testing 3 genes...

In cluster Multiciliated lineage:
False    26913
True     10617
Name: smoking_status, dtype: int64
female    21902
male      15628
Name: sex, dtype: int64
Testing 3 genes...

In clus

## Inspect some results

In [46]:
de_results_lvl3_glm.keys()

dict_keys(['Macrophages', 'Basal', 'AT2', 'T cell lineage', 'Monocytes', 'Multiciliated lineage', 'Submucosal Secretory', 'AT1', 'Innate lymphoid cells', 'Secretory', 'Capillary', 'Mast cells', 'B cell lineage', 'Fibroblasts', 'Dendritic cells', 'Venous', 'Lymphatic EC', 'Arterial', 'Rare', 'Airway smooth muscle'])

In [47]:
full_res_lvl3_glm

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr NS,0.815131,0.363377,0.024883,0.049766
1,ACE2,Macrophages,sex[T.male],31yr S,0.103947,0.766362,0.892108,0.917597
2,ACE2,Macrophages,sex[T.male],62yr NS,-0.621819,0.501752,0.215236,0.286981
3,ACE2,Macrophages,sex[T.male],62yr S,-1.333003,0.584259,0.022517,0.047683
4,ACE2,Macrophages,age,F NS,0.021959,0.009962,0.027511,0.052126
...,...,...,...,...,...,...,...,...
583,CTSL,Airway smooth muscle,age,M S,-0.017410,0.007137,0.014716,0.110850
584,CTSL,Airway smooth muscle,smoking_status[T.True],F 31yr,3.599766,2.264954,0.111985,0.335956
585,CTSL,Airway smooth muscle,smoking_status[T.True],F 62yr,-0.360364,0.480425,0.453197,0.549023
586,CTSL,Airway smooth muscle,smoking_status[T.True],M 31yr,2.702614,1.147140,0.018475,0.110850


In [48]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'ACE2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr NS,0.815131,0.363377,2.488309e-02,4.976618e-02
1,ACE2,Macrophages,sex[T.male],31yr S,0.103947,0.766362,8.921080e-01,9.175968e-01
2,ACE2,Macrophages,sex[T.male],62yr NS,-0.621819,0.501752,2.152359e-01,2.869813e-01
3,ACE2,Macrophages,sex[T.male],62yr S,-1.333003,0.584259,2.251720e-02,4.768348e-02
4,ACE2,Macrophages,age,F NS,0.021959,0.009962,2.751099e-02,5.212609e-02
5,ACE2,Macrophages,age,F S,0.055118,0.023564,1.932993e-02,4.349235e-02
6,ACE2,Macrophages,age,M NS,-0.024394,0.014931,1.023106e-01,1.492460e-01
7,ACE2,Macrophages,age,M S,0.008765,0.017944,6.252326e-01,7.355086e-01
8,ACE2,Macrophages,smoking_status[T.True],F 31yr,-0.045619,0.829895,9.561629e-01,9.561629e-01
9,ACE2,Macrophages,smoking_status[T.True],F 62yr,0.982299,0.530255,6.395348e-02,1.096345e-01


In [49]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'ACE2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr NS,0.815131,0.363377,2.488309e-02,4.976618e-02
3,ACE2,Macrophages,sex[T.male],62yr S,-1.333003,0.584259,2.251720e-02,4.768348e-02
5,ACE2,Macrophages,age,F S,0.055118,0.023564,1.932993e-02,4.349235e-02
37,ACE2,Basal,sex[T.male],31yr S,0.256239,0.090028,4.424246e-03,6.125879e-03
39,ACE2,Basal,sex[T.male],62yr S,0.298193,0.076806,1.034219e-04,1.692358e-04
40,ACE2,Basal,age,F NS,0.004779,0.002113,2.371321e-02,2.845585e-02
41,ACE2,Basal,age,F S,-0.011966,0.003272,2.550366e-04,3.991877e-04
42,ACE2,Basal,age,M NS,0.006132,0.001808,6.925003e-04,1.038751e-03
43,ACE2,Basal,age,M S,-0.010613,0.003261,1.134809e-03,1.634125e-03
44,ACE2,Basal,smoking_status[T.True],F 31yr,0.408095,0.099383,4.020557e-05,7.237003e-05


In [50]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
12,TMPRSS2,Macrophages,sex[T.male],31yr NS,-0.165214,0.099626,9.724890e-02,1.492460e-01
13,TMPRSS2,Macrophages,sex[T.male],31yr S,-0.392816,0.133601,3.279884e-03,8.433989e-03
14,TMPRSS2,Macrophages,sex[T.male],62yr NS,-0.392189,0.152453,1.009587e-02,2.423008e-02
15,TMPRSS2,Macrophages,sex[T.male],62yr S,-0.619791,0.152050,4.577273e-05,1.267552e-04
16,TMPRSS2,Macrophages,age,F NS,-0.000448,0.002654,8.659952e-01,9.169361e-01
17,TMPRSS2,Macrophages,age,F S,0.001792,0.006504,7.828606e-01,8.807182e-01
18,TMPRSS2,Macrophages,age,M NS,-0.007770,0.004774,1.036430e-01,1.492460e-01
19,TMPRSS2,Macrophages,age,M S,-0.005529,0.004996,2.683731e-01,3.450512e-01
20,TMPRSS2,Macrophages,smoking_status[T.True],F 31yr,0.256512,0.149999,8.724778e-02,1.427691e-01
21,TMPRSS2,Macrophages,smoking_status[T.True],F 62yr,0.325956,0.153126,3.328078e-02,5.990541e-02


In [51]:
full_res_lvl3_glm.loc[(full_res_lvl3_glm['gene'] == 'TMPRSS2') & (full_res_lvl3_glm['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
13,TMPRSS2,Macrophages,sex[T.male],31yr S,-0.392816,0.133601,3.279884e-03,8.433989e-03
14,TMPRSS2,Macrophages,sex[T.male],62yr NS,-0.392189,0.152453,1.009587e-02,2.423008e-02
15,TMPRSS2,Macrophages,sex[T.male],62yr S,-0.619791,0.152050,4.577273e-05,1.267552e-04
48,TMPRSS2,Basal,sex[T.male],31yr NS,0.410298,0.026951,0.000000e+00,0.000000e+00
49,TMPRSS2,Basal,sex[T.male],31yr S,0.940702,0.043136,0.000000e+00,0.000000e+00
51,TMPRSS2,Basal,sex[T.male],62yr S,0.491168,0.039338,0.000000e+00,0.000000e+00
52,TMPRSS2,Basal,age,F NS,0.015555,0.000868,0.000000e+00,0.000000e+00
53,TMPRSS2,Basal,age,F S,-0.006559,0.001658,7.643514e-05,1.310317e-04
55,TMPRSS2,Basal,age,M S,-0.021060,0.001553,0.000000e+00,0.000000e+00
56,TMPRSS2,Basal,smoking_status[T.True],F 31yr,-0.133554,0.049944,7.494061e-03,9.635221e-03


In [52]:
full_res_lvl3_glm.loc[full_res_lvl3_glm['gene'] == 'CTSL',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
24,CTSL,Macrophages,sex[T.male],31yr NS,-0.208574,0.003866,0.000000e+00,0.000000e+00
25,CTSL,Macrophages,sex[T.male],31yr S,0.145658,0.005647,0.000000e+00,0.000000e+00
26,CTSL,Macrophages,sex[T.male],62yr NS,-0.279511,0.004790,0.000000e+00,0.000000e+00
27,CTSL,Macrophages,sex[T.male],62yr S,0.074720,0.006310,0.000000e+00,0.000000e+00
28,CTSL,Macrophages,age,F NS,0.005645,0.000110,0.000000e+00,0.000000e+00
29,CTSL,Macrophages,age,F S,-0.005114,0.000231,0.000000e+00,0.000000e+00
30,CTSL,Macrophages,age,M NS,0.003357,0.000169,0.000000e+00,0.000000e+00
31,CTSL,Macrophages,age,M S,-0.007403,0.000153,0.000000e+00,0.000000e+00
32,CTSL,Macrophages,smoking_status[T.True],F 31yr,0.105873,0.006549,0.000000e+00,0.000000e+00
33,CTSL,Macrophages,smoking_status[T.True],F 62yr,-0.227678,0.006604,0.000000e+00,0.000000e+00


In [53]:
#No smoking model
full_res_lvl3_glm_nosmoke.loc[full_res_lvl3_glm_nosmoke['gene'] == 'ACE2',]
full_res_lvl3_glm_nosmoke.loc[full_res_lvl3_glm_nosmoke['gene'] == 'TMPRSS2',]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
0,ACE2,Macrophages,sex[T.male],31yr,0.644219,0.349797,6.551964e-02,9.827946e-02
1,ACE2,Macrophages,sex[T.male],62yr,-0.577714,0.347581,9.649363e-02,1.157924e-01
2,ACE2,Macrophages,age,F,0.023746,0.009730,1.466525e-02,2.514043e-02
3,ACE2,Macrophages,age,M,-0.015671,0.010974,1.532856e-01,1.672207e-01
12,ACE2,Basal,sex[T.male],31yr,-0.046537,0.059813,4.365482e-01,4.762345e-01
13,ACE2,Basal,sex[T.male],62yr,0.103143,0.061120,9.149816e-02,1.097978e-01
14,ACE2,Basal,age,F,0.001302,0.001921,4.979171e-01,4.979171e-01
15,ACE2,Basal,age,M,0.006130,0.001642,1.891067e-04,2.836601e-04
24,ACE2,AT2,sex[T.male],31yr,0.963017,0.072970,0.000000e+00,0.000000e+00
25,ACE2,AT2,sex[T.male],62yr,0.750407,0.079957,0.000000e+00,0.000000e+00


,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
4,TMPRSS2,Macrophages,sex[T.male],31yr,-0.215222,0.086376,1.271411e-02,2.514043e-02
5,TMPRSS2,Macrophages,sex[T.male],62yr,-0.377470,0.097364,1.058039e-04,2.539293e-04
6,TMPRSS2,Macrophages,age,F,-0.000516,0.002569,8.408546e-01,8.408546e-01
7,TMPRSS2,Macrophages,age,M,-0.005750,0.003276,7.925235e-02,1.056698e-01
16,TMPRSS2,Basal,sex[T.male],31yr,0.458940,0.025607,0.000000e+00,0.000000e+00
17,TMPRSS2,Basal,sex[T.male],62yr,0.157276,0.026020,1.499622e-09,2.570780e-09
18,TMPRSS2,Basal,age,F,0.010933,0.000858,0.000000e+00,0.000000e+00
19,TMPRSS2,Basal,age,M,0.001202,0.000591,4.197791e-02,5.597055e-02
28,TMPRSS2,AT2,sex[T.male],31yr,0.312871,0.011219,0.000000e+00,0.000000e+00
29,TMPRSS2,AT2,sex[T.male],62yr,-0.177075,0.008840,0.000000e+00,0.000000e+00


In [54]:
# No smoking model
full_res_lvl3_glm_nosmoke.loc[(full_res_lvl3_glm_nosmoke['gene'] == 'ACE2') & (full_res_lvl3_glm_nosmoke['adj_pvals'] < 0.05),]

,gene,cell_identity,covariate,eval_at,coef,coef_sd,pval,adj_pvals
2,ACE2,Macrophages,age,F,0.023746,0.009730,1.466525e-02,2.514043e-02
15,ACE2,Basal,age,M,0.006130,0.001642,1.891067e-04,2.836601e-04
24,ACE2,AT2,sex[T.male],31yr,0.963017,0.072970,0.000000e+00,0.000000e+00
25,ACE2,AT2,sex[T.male],62yr,0.750407,0.079957,0.000000e+00,0.000000e+00
26,ACE2,AT2,age,F,0.013807,0.001712,7.771561e-16,1.036208e-15
27,ACE2,AT2,age,M,0.006949,0.003147,2.726132e-02,2.973963e-02
48,ACE2,Monocytes,sex[T.male],31yr,1.328918,0.635962,3.665196e-02,4.886928e-02
60,ACE2,Multiciliated lineage,sex[T.male],31yr,0.369222,0.107082,5.646557e-04,1.129311e-03
62,ACE2,Multiciliated lineage,age,F,0.011613,0.003428,7.033263e-04,1.205702e-03
75,ACE2,Submucosal Secretory,age,M,0.010597,0.003481,2.329719e-03,3.993804e-03


# Store results

In [56]:
full_res_lvl2_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl2_full.csv')

In [57]:
full_res_lvl3_glm.to_csv(folder+'/'+output_folder+de_output_base+'_lvl3_full.csv')